In [ ]:
import pandas as pd
# from sklearn import model_selection as ms
import json
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import RidgeClassifier,LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
import numpy as np
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/sent_use_690k.csv')

In [ ]:
df_vp = df[df['label'] == 5].sample(1000).reset_index(drop=True)
df_p = df[df['label'] == 4].sample(1000).reset_index(drop=True)
df_ne = df[df['label'] == 3].sample(1000).reset_index(drop=True)
df_n = df[df['label'] == 2].sample(1000).reset_index(drop=True)
df_vn = df[df['label'] == 1].sample(1000).reset_index(drop=True)

In [ ]:
df = pd.concat([df_vp,df_p,df_ne,df_n,df_vn],ignore_index=True).sample(frac=1).reset_index(drop=True)

In [ ]:
df

,id,label,embeddings,kfold
0,eed774e7dfc383e8c00f7c49f64cb5fd9ae08f64f6c1ee...,5.0,"[-0.009865188971161842,-0.0048142122104763985,...",46.0
1,7ea38247a240d90bf87a9cf0cad74de42c117a617d4e6c...,3.0,"[0.004865941125899553,0.010130273178219795,-0....",48.0
2,93aabf958a39e1736e8151083a21b1c040fd83fb410dcf...,1.0,"[0.062471870332956314,0.06932082772254944,-0.0...",48.0
3,60276591fcd54b589a4d972b359f26735fbd0ab29a67d6...,2.0,"[0.04490180313587189,0.04536549374461174,-0.03...",45.0
4,4934aca5afba6e85e13d7f5e09222d8ee8fec0770f01fe...,5.0,"[0.007310839835554361,0.0029883820097893476,0....",45.0
...,...,...,...,...
4995,ad7f09c8df1a306228ef996d398e50d70382b645aa6ac0...,5.0,"[-0.012337270192801952,0.022855741903185844,-0...",46.0
4996,cd74de741185b0b5b74248a86f11b2ef8b4463f4f74878...,5.0,"[-0.01864706352353096,-0.010123437270522118,0....",48.0
4997,fd0e6efcda9f51d5d1273fba36f2d8ef8d02dd942df574...,5.0,"[-0.013296646066009998,-0.04930303618311882,-0...",48.0
4998,e0566c8e0d14386425b089eae8efe23a34a7a3312cb8dc...,1.0,"[-0.08055581897497177,0.01832742802798748,0.01...",46.0


In [ ]:
skf = RepeatedStratifiedKFold(n_splits=2)
for f,(t_,v_) in enumerate(skf.split(X=df,y=df.label.values)):
    print(f)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [ ]:
df = df.drop('Unnamed: 0',axis = 1)
df = df.rename({'USE_embed':'embeddings'},axis=1)

In [ ]:
df_train = []
df_val = []
for i in range(5):
  df_train.append(df[df['kfold']!=i].reset_index(drop=True))
  df_val.append(df[df['kfold']==i].reset_index(drop=True))

In [ ]:
def prepare_data(i):
    
    mat_tr = np.matrix([x for x in  df_train[i]['embeddings'].apply(lambda x: json.loads(x))])
    mat_ts = np.matrix([x for x in  df_val[i]['embeddings'].apply(lambda x: json.loads(x))])
    x_tr= np.squeeze(np.asarray(mat_tr))
    x_ts= np.squeeze(np.asarray(mat_ts))
    y_tr = df_train[i].label.values
    y_ts = df_val[i].label.values
    return x_tr,y_tr,x_ts,y_ts

In [ ]:
x = df.iloc[0]['embeddings']

In [ ]:
type(json.loads(x)[0])

list

In [ ]:
class Model:
  def __init__(self,file,models):
    self.models = models
    df = pd.read_csv(file)
    df = df.drop('Unnamed: 0',axis = 1)
    df = df.rename({'USE_embed':'embeddings'},axis=1)
    df_vp = df[df['label'] == 5].sample(1000).reset_index(drop=True)
    df_p = df[df['label'] == 4].sample(1000).reset_index(drop=True)
    df_ne = df[df['label'] == 3].sample(1000).reset_index(drop=True)
    df_n = df[df['label'] == 2].sample(1000).reset_index(drop=True)
    df_vn = df[df['label'] == 1].sample(1000).reset_index(drop=True)
    df = pd.concat([df_vp,df_p,df_ne,df_n,df_vn],ignore_index=True).sample(frac=1).reset_index(drop=True)
    skf = StratifiedKFold(n_splits=5)
    for f,(t_,v_) in enumerate(skf.split(X=df,y=df.label.values)):
       df.loc[v_,'kfold'] =f
    self.dfs_train = [df[df.kfold != f].reset_index(drop=True) for f in range(0,5)]
    self.dfs_valid = [df[df.kfold == f].reset_index(drop=True) for f in range(0,5)]
  

  def prepare_data(self,i):
    
    mat_tr = np.matrix([x for x in  self.dfs_train[i]['embeddings'].apply(lambda x: json.loads(x))])
    mat_ts = np.matrix([x for x in  self.dfs_valid[i]['embeddings'].apply(lambda x: json.loads(x))])
    x_tr= np.squeeze(np.asarray(mat_tr))
    x_ts= np.squeeze(np.asarray(mat_ts))
    y_tr = self.dfs_train[i].label.values
    y_ts = self.dfs_valid[i].label.values
    return x_tr,y_tr,x_ts,y_ts
  
  def run_models(self):
    self.all = {}
    for mod in self.models:
      self.all[str(mod)] = self.pass_model(mod)
    
  def get_all(self):
    return self.all
    
  def pass_model(self,mod):
    acc = []
    for i in range(0,len(self.dfs_train)):
      x_tr,y_tr,x_ts,y_ts = self.prepare_data(i)
      acc.append(self.train(mod,x_tr,y_tr,x_ts,y_ts))
    return acc

  def train(self,model,x_tr,y_tr,x_ts,y_ts):
    if model == MLPClassifier:
      mod = model(early_stopping =True)
    else:
      mod = model()

    mod = OneVsRestClassifier(mod)
    mod.fit(x_tr,y_tr)
    return accuracy_score(y_ts,mod.predict(x_ts))


In [ ]:
m = Model('/content/drive/MyDrive/sent_use_690k.csv',[LogisticRegression,GaussianNB,MLPClassifier,RidgeClassifier])

In [ ]:
m.run_models()

In [ ]:
x = m.get_all()

In [ ]:
with open('/content/drive/MyDrive/sent_use_ovr_accuracy.txt','w') as f:
  f.write(str(x))

In [ ]:
dfs_train = [df[df.kfold != f].reset_index(drop=True) for f in range(0,5)]
dfs_valid = [df[df.kfold == f].reset_index(drop=True) for f in range(0,5)]
  

In [ ]:
skf = RepeatedStratifiedKFold(n_splits=5)
for f,(t_,v_) in enumerate(skf.split(X=df,y=df.label.values)):
    df.loc[v_,'kfold'] =f

In [ ]:
m = Model('/content/drive/MyDrive/sar_use_data.csv',[GaussianNB,MLPClassifier])

FileNotFoundError: ignored

In [ ]:
with open('/content/drive/MyDrive/sarcasm_use_accuracy.txt') as f:
  f.write(m.get_all())

In [ ]:
x = prepare_data(0)

In [ ]:
x1,y1,x2,y2 = x

In [ ]:
x2.shape

(0,)